In [64]:
# Import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

In [65]:
# Load the datasets
customers_df = pd.read_csv('Customers.csv')
products_df = pd.read_csv('Products.csv')
transactions_df = pd.read_csv('Transactions.csv')

In [66]:
# Clean column names by stripping any leading/trailing spaces
customers_df.columns = customers_df.columns.str.strip()
products_df.columns = products_df.columns.str.strip()
transactions_df.columns = transactions_df.columns.str.strip()

# Display the first few rows of each dataset to verify the data
print(customers_df.head())
print(products_df.head())
print(transactions_df.head())

  CustomerID        CustomerName         Region  SignupDate
0      C0001    Lawrence Carroll  South America  2022-07-10
1      C0002      Elizabeth Lutz           Asia  2022-02-13
2      C0003      Michael Rivera  South America  2024-03-07
3      C0004  Kathleen Rodriguez  South America  2022-10-09
4      C0005         Laura Weber           Asia  2022-08-15
  ProductID              ProductName     Category   Price
0      P001     ActiveWear Biography        Books  169.30
1      P002    ActiveWear Smartwatch  Electronics  346.30
2      P003  ComfortLiving Biography        Books   44.12
3      P004            BookWorld Rug   Home Decor   95.69
4      P005          TechPro T-Shirt     Clothing  429.31
  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3       

In [67]:
# Check if the 'Category' column exists in products_df
print("Columns in products_df:", products_df.columns)

# Merge transactions with product information
transactions_df = transactions_df.merge(products_df, on='ProductID', how='left')

# Check if the 'Category' column was successfully merged
print("Columns after merge:", transactions_df.columns)

Columns in products_df: Index(['ProductID', 'ProductName', 'Category', 'Price'], dtype='object')
Columns after merge: Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_x', 'ProductName', 'Category',
       'Price_y'],
      dtype='object')


In [68]:
# Prepare customer profile data (only relevant columns)
customer_profile = customers_df[['CustomerID', 'Region', 'SignupDate']]

# Convert 'SignupDate' to datetime format
customer_profile['SignupDate'] = pd.to_datetime(customer_profile['SignupDate'])

# Display the customer profile to verify the data
print(customer_profile.head())

  CustomerID         Region SignupDate
0      C0001  South America 2022-07-10
1      C0002           Asia 2022-02-13
2      C0003  South America 2024-03-07
3      C0004  South America 2022-10-09
4      C0005           Asia 2022-08-15


C:\Users\shekh\AppData\Local\Temp\ipykernel_17276\1487893007.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_profile['SignupDate'] = pd.to_datetime(customer_profile['SignupDate'])


In [69]:
# Aggregate transaction data for each customer
customer_transactions = transactions_df.groupby('CustomerID').agg(
    total_spent=('TotalValue', 'sum'),
    num_transactions=('TransactionID', 'count')
).reset_index()

# Merge the aggregated transaction data with the customer profile
customer_profile = customer_profile.merge(customer_transactions, on='CustomerID', how='left')

# Check for missing values after merge
print(customer_profile.isnull().sum())

# Fill NaN values with 0 for 'total_spent' and 'num_transactions' if any missing
customer_profile['total_spent'].fillna(0, inplace=True)
customer_profile['num_transactions'].fillna(0, inplace=True)

# Display the updated customer profile after filling missing values
print(customer_profile.head())

CustomerID          0
Region              0
SignupDate          0
total_spent         1
num_transactions    1
dtype: int64
  CustomerID         Region SignupDate  total_spent  num_transactions
0      C0001  South America 2022-07-10      3354.52               5.0
1      C0002           Asia 2022-02-13      1862.74               4.0
2      C0003  South America 2024-03-07      2725.38               4.0
3      C0004  South America 2022-10-09      5354.88               8.0
4      C0005           Asia 2022-08-15      2034.24               3.0


C:\Users\shekh\AppData\Local\Temp\ipykernel_17276\1698593543.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer_profile['total_spent'].fillna(0, inplace=True)
C:\Users\shekh\AppData\Local\Temp\ipykernel_17276\1698593543.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

In [70]:
# Convert 'Region' to a numerical code
customer_profile['Region'] = customer_profile['Region'].astype('category').cat.codes

# Add a 'signup_year' feature extracted from the 'SignupDate'
customer_profile['signup_year'] = customer_profile['SignupDate'].dt.year

# Display the customer profile after feature engineering
print(customer_profile.head())


  CustomerID  Region SignupDate  total_spent  num_transactions  signup_year
0      C0001       3 2022-07-10      3354.52               5.0         2022
1      C0002       0 2022-02-13      1862.74               4.0         2022
2      C0003       3 2024-03-07      2725.38               4.0         2024
3      C0004       3 2022-10-09      5354.88               8.0         2022
4      C0005       0 2022-08-15      2034.24               3.0         2022


In [71]:
# Select features for similarity calculation
features = customer_profile[['Region', 'total_spent', 'num_transactions', 'signup_year']]

# Normalize the features using StandardScaler
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Display the normalized features
print(features_scaled[:5])  # Display the first 5 rows


[[ 1.24138358 -0.05188436  0.         -1.27635218]
 [-1.40925752 -0.86271433 -0.45129368 -1.27635218]
 [ 1.24138358 -0.393842   -0.45129368  1.09825653]
 [ 1.24138358  1.03537505  1.35388105 -1.27635218]
 [-1.40925752 -0.76949861 -0.90258736 -1.27635218]]


In [72]:
# Compute the cosine similarity matrix
similarity_matrix = cosine_similarity(features_scaled)

# Display the similarity matrix shape
print(similarity_matrix.shape)


(200, 200)


In [73]:
# Function to get top 3 lookalike customers based on cosine similarity
def get_top_lookalikes(customer_id, similarity_matrix, customer_profile):
    # Find the index of the customer_id
    customer_index = customer_profile[customer_profile['CustomerID'] == customer_id].index[0]
    
    # Get the similarity scores for the customer
    similarity_scores = similarity_matrix[customer_index]
    
    # Create a list of customer ids with similarity scores
    similar_customers = []
    for i, score in enumerate(similarity_scores):
        if i != customer_index:  # Exclude the customer itself
            similar_customers.append((customer_profile.iloc[i]['CustomerID'], score))
    
    # Sort by similarity score (highest first) and take top 3
    similar_customers.sort(key=lambda x: x[1], reverse=True)
    return similar_customers[:3]

# Display top 3 lookalikes for the first customer (C0001)
top_lookalikes = get_top_lookalikes('C0001', similarity_matrix, customer_profile)
print(top_lookalikes)


[('C0152', 0.999954289194027), ('C0011', 0.9934625127648987), ('C0174', 0.9902719390028006)]


In [74]:
# Generate top 3 lookalikes for the first 20 customers
lookalikes_dict = {}
for customer_id in customer_profile['CustomerID'][:20]:
    lookalikes_dict[customer_id] = get_top_lookalikes(customer_id, similarity_matrix, customer_profile)

# Display the lookalikes for the first customer
print(lookalikes_dict['C0001'])


[('C0152', 0.999954289194027), ('C0011', 0.9934625127648987), ('C0174', 0.9902719390028006)]


In [75]:
# Prepare the data for export
lookalike_data = []
for customer_id, lookalikes in lookalikes_dict.items():
    for lookalike, score in lookalikes:
        lookalike_data.append([customer_id, lookalike, score])

# Convert the list to a DataFrame
lookalike_df = pd.DataFrame(lookalike_data, columns=['CustomerID', 'LookalikeCustomerID', 'SimilarityScore'])

# Display the first few rows of the lookalike DataFrame
print(lookalike_df.head())


  CustomerID LookalikeCustomerID  SimilarityScore
0      C0001               C0152         0.999954
1      C0001               C0011         0.993463
2      C0001               C0174         0.990272
3      C0002               C0027         0.984926
4      C0002               C0005         0.978967


In [77]:
# Save the lookalike results to a CSV file
lookalike_df.to_csv('Shekhar_Yadav_Lookalike.csv', index=False)

# Notify user that the CSV has been saved
print("Lookalike recommendations saved to 'Lookalike.csv'")

Lookalike recommendations saved to 'Lookalike.csv'
